<a href="https://colab.research.google.com/github/Muzanua/CapstoneProject/blob/main/modelTrain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd

df = pd.read_csv('plant_health_data.csv')
print(df.columns)

Index(['Timestamp', 'Plant_ID', 'Soil_Moisture', 'Ambient_Temperature',
       'Soil_Temperature', 'Humidity', 'Light_Intensity', 'Soil_pH',
       'Nitrogen_Level', 'Phosphorus_Level', 'Potassium_Level',
       'Chlorophyll_Content', 'Electrochemical_Signal', 'Plant_Health_Status'],
      dtype='object')


In [ ]:
print(df.describe())
print(df[['Soil_Moisture', 'Ambient_Temperature', 'Humidity']].corr())

          Plant_ID  Soil_Moisture  Ambient_Temperature  Soil_Temperature  \
count  1200.000000    1200.000000          1200.000000       1200.000000   
mean      5.500000      25.106918            23.999130         19.957794   
std       2.873479       8.677725             3.441561          2.932073   
min       1.000000      10.000724            18.001993         15.003710   
25%       3.000000      17.131893            21.101766         17.353027   
50%       5.500000      25.168333            23.889044         19.911473   
75%       8.000000      32.370231            27.042634         22.596851   
max      10.000000      39.993164            29.990886         24.995929   

          Humidity  Light_Intensity      Soil_pH  Nitrogen_Level  \
count  1200.000000      1200.000000  1200.000000     1200.000000   
mean     54.853165       612.637265     6.524102       30.106751   
std       8.784916       228.318853     0.581755       11.514396   
min      40.028758       200.615482     5.5

In [ ]:
# ==========================================
# STEP 1: Import Library
# ==========================================
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler, LabelEncoder
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
import tensorflow as tf
from tensorflow.keras import layers

# ==========================================
# STEP 2: Load Data
# ==========================================
df = pd.read_csv('plant_health_data.csv')

# Pilih fitur dan target
X = df[['Soil_Moisture', 'Ambient_Temperature', 'Humidity']].values
y = df['Plant_Health_Status'].values  # Kategorikal

# ==========================================
# STEP 3: Preprocessing Label
# ==========================================
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)       # 'Healthy' → 0, dst
y_categorical = to_categorical(y_encoded, num_classes=3)

# ==========================================
# STEP 4: Normalisasi Input
# ==========================================
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)

# ==========================================
# STEP 5: Train/Val/Test Split
# ==========================================
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y_categorical, test_size=0.3, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

# ==========================================
# STEP 6: Build Model
# ==========================================
def build_model(input_shape, num_classes):
    model = tf.keras.Sequential([
        tf.keras.Input(shape=input_shape),
        layers.Dense(32, activation='relu'),
        layers.Dense(16, activation='relu'),
        layers.Dense(num_classes, activation='softmax')
    ])
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

model = build_model((3,), 3)

# ==========================================
# STEP 7: Train
# ==========================================
model.fit(X_train, y_train, epochs=50, batch_size=16, validation_data=(X_val, y_val), verbose=1)

# ==========================================
# STEP 8: Convert to TFLite
# ==========================================
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

with open("PlantHealth.tflite", "wb") as f:
    f.write(tflite_model)

# ==========================================
# STEP 9: Convert to .h
# ==========================================
!xxd -i PlantHealth.tflite > PlantHealth.h


Epoch 1/50
42/42 ━━━━━━━━━━━━━━━━━━━━ 5s 10ms/step - accuracy: 0.3056 - loss: 1.0993 - val_accuracy: 0.5833 - val_loss: 1.0663
Epoch 2/50
42/42 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.6023 - loss: 1.0562 - val_accuracy: 0.6071 - val_loss: 1.0300
Epoch 3/50
42/42 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.6209 - loss: 1.0149 - val_accuracy: 0.5833 - val_loss: 0.9769
Epoch 4/50
42/42 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.6061 - loss: 0.9412 - val_accuracy: 0.5952 - val_loss: 0.9141
Epoch 5/50
42/42 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.6239 - loss: 0.8750 - val_accuracy: 0.6131 - val_loss: 0.8621
Epoch 6/50
42/42 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.6119 - loss: 0.8496 - val_accuracy: 0.6429 - val_loss: 0.8151
Epoch 7/50
42/42 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.6880 - loss: 0.7726 - val_accuracy: 0.6548 - val_loss: 0.7643
Epoch 8/50
42/42 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7286 - loss: 0.7186 - val_accuracy: 0.6726 - val_loss

In [ ]:
# Cek hasil
print(list(zip(y, y_encoded)))

[('High Stress', np.int64(1)), ('High Stress', np.int64(1)), ('High Stress', np.int64(1)), ('High Stress', np.int64(1)), ('High Stress', np.int64(1)), ('High Stress', np.int64(1)), ('High Stress', np.int64(1)), ('Moderate Stress', np.int64(2)), ('High Stress', np.int64(1)), ('High Stress', np.int64(1)), ('Moderate Stress', np.int64(2)), ('Healthy', np.int64(0)), ('Moderate Stress', np.int64(2)), ('Moderate Stress', np.int64(2)), ('Moderate Stress', np.int64(2)), ('Moderate Stress', np.int64(2)), ('Moderate Stress', np.int64(2)), ('High Stress', np.int64(1)), ('Moderate Stress', np.int64(2)), ('Moderate Stress', np.int64(2)), ('Healthy', np.int64(0)), ('Healthy', np.int64(0)), ('Healthy', np.int64(0)), ('Healthy', np.int64(0)), ('Healthy', np.int64(0)), ('Moderate Stress', np.int64(2)), ('High Stress', np.int64(1)), ('Healthy', np.int64(0)), ('High Stress', np.int64(1)), ('High Stress', np.int64(1)), ('High Stress', np.int64(1)), ('Healthy', np.int64(0)), ('Healthy', np.int64(0)), ('Hig